In [1]:
import os

In [2]:
%pwd

'e:\\Text-Summarizer-Project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'e:\\Text-Summarizer-Project'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: Path

In [6]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            tokenizer_name = config.tokenizer_name
        )

        return data_transformation_config

In [8]:
#creating components
import os
from textSummarizer.logging import logger
import pandas as pd
from transformers import AutoTokenizer
from datasets import load_dataset, load_from_disk

c:\Users\Rashi Kathuria\anaconda3\envs\textS\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2024-05-17 03:14:47,931: INFO: config: PyTorch version 2.3.0 available.]


In [9]:
# class DataTransformation:
#     def __init__(self, config: DataTransformationConfig):
#         self.config = config
#         self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)


    
#     def convert_examples_to_features(self,example_batch):
#         input_encodings = self.tokenizer(example_batch['article'] , max_length = 1024, truncation = True )
        
#         with self.tokenizer.as_target_tokenizer():
#             target_encodings = self.tokenizer(example_batch['abstract'], max_length = 128, truncation = True )
            
#         return {
#             'input_ids' : input_encodings['input_ids'],
#             'attention_mask': input_encodings['attention_mask'],
#             'labels': target_encodings['input_ids']
#         }
    

#     def convert(self):
        
#         # Check if the dataset path exists
#         if not os.path.exists(self.config.data_path):
#             raise FileNotFoundError(f"Directory {self.config.data_path} does not exist.")
#         dataset_samsum = load_from_disk(self.config.data_path)
#         dataset_samsum_pt = dataset_samsum.map(self.convert_examples_to_features, batched = True)
#         dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir,"dataset"))

# import os
# from datasets import Dataset, DatasetDict, Features, Value
# import pandas as pd
# from transformers import AutoTokenizer




In [10]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)
    
    def convert_examples_to_features(self, example_batch):
        input_encodings = self.tokenizer(example_batch['article'] , max_length = 1024, truncation = True )

        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(example_batch['abstract'], max_length = 128, truncation = True )

        return {
            'input_ids' : input_encodings['input_ids'],
            'attention_mask': input_encodings['attention_mask'],
            'labels': target_encodings['input_ids']
        }
    

    def convert(self):
        dataset = load_from_disk(self.config.data_path)
        print("Dataset loaded")
        dataset_pt = dataset.map(self.convert_examples_to_features, batched = True)
        dataset_pt.save_to_disk(os.path.join(self.config.root_dir,"dataset"))

In [11]:
#creating pipeline
# try:
#     config = ConfigurationManager()
#     data_transformation_config = config.get_data_transformation_config()
#     data_transformation = DataTransformation(config=data_transformation_config)
#     data_transformation.convert()
# except Exception as e:
#     raise e



In [10]:
import os
from datasets import Dataset, DatasetDict, Features, Value
import pandas as pd
from transformers import AutoTokenizer

class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)

    def convert_examples_to_features(self, examples):
        # Ensure 'article' and 'abstract' are strings
        articles = [str(article) for article in examples['article']]
        abstracts = [str(abstract) for abstract in examples['abstract']]

        # Tokenize with truncation and padding
        inputs = self.tokenizer(
            articles,
            abstracts,
            max_length=1024,
            truncation=True,
            padding='max_length'
        )
        return inputs

    def convert(self):
        # Load dataset folder
        dataset_path = self.config.data_path
        train_df = pd.read_csv(os.path.join(dataset_path, 'train', 'train.csv'))
        test_df = pd.read_csv(os.path.join(dataset_path, 'test', 'test.csv'))
        val_df = pd.read_csv(os.path.join(dataset_path, 'validation', 'validation.csv'))

        # Optionally handle missing values (if needed)
        # train_df['article'].fillna("EMPTY", inplace=True)
        # train_df['abstract'].fillna("EMPTY", inplace=True)
        # Repeat for test_df and val_df

        # Define features
        features = Features({
            "article": Value("string"),
            "abstract": Value("string")
        })

        # Create datasets from DataFrames
        train_dataset = Dataset.from_pandas(train_df, features=features)
        test_dataset = Dataset.from_pandas(test_df, features=features)
        val_dataset = Dataset.from_pandas(val_df, features=features)

        # Create DatasetDict
        dataset = DatasetDict({
            "train": train_dataset,
            "test": test_dataset,
            "validation": val_dataset
        })

        # Map each dataset to features
        dataset = dataset.map(self.convert_examples_to_features, batched=True)

        # Save processed datasets
        output_path = os.path.join(self.config.root_dir, "dataset")
        os.makedirs(output_path, exist_ok=True)
        dataset.save_to_disk(output_path)
        print("Dataset saved successfully!")

try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.convert()
except Exception as e:
    print(f"An error occurred: {e}")


[2024-05-17 03:21:50,212: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-05-17 03:21:50,216: INFO: common: yaml file: params.yaml loaded successfully]
[2024-05-17 03:21:50,217: INFO: common: created directory at: artifacts]
[2024-05-17 03:21:50,219: INFO: common: created directory at: artifacts/data_transformation]


c:\Users\Rashi Kathuria\anaconda3\envs\textS\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Saving the dataset (1/1 shards): 100%|██████████| 600/600 [00:00<00:00, 38002.78 examples/s]


Dataset saved successfully!
